In [1]:
def counter():
    num = 0
    def incrementer():
        nonlocal num
        num += 1
        return num
    return incrementer

In [2]:
c = counter()
print(c, c())
c()

<function counter.<locals>.incrementer at 0x0000013D41AA6378> 1


2

In [3]:
print(counter())
counter()()

<function counter.<locals>.incrementer at 0x0000013D41AA68C8>


1

In [4]:
print(c())
print(counter()())

3
1


In [5]:
x = 'Hi'

def read_x():
    print(x)
    
def read_y():
    print(y)
    
read_x()
read_y()

Hi


NameError: name 'y' is not defined

In [6]:
def read_y():
    y = 'Hey'
    print(y)
    
read_y()

Hey


In [7]:
def read_x_local_fail():
    if False:
        x = 'Hey'  # x appears in an assignment, therefore it's local
    print(x)       # will look for the _local_ z, which is not assigned, and will not be found
    
read_x_local_fail()  # UnboundLocalError: local variable 'x' referenced before assignment

UnboundLocalError: local variable 'x' referenced before assignment

In [8]:
def change_local_x():
    x = 'Bye'
    print(x)
    
change_local_x()
print(x)

Bye
Hi


In [9]:
def change_global_x():
    global x
    x = 'Bye'
    print(x)
    
change_global_x()
print(x)

Bye
Bye


1. if you've found `global x`, then x is a **global** variable
2. If you've found `nonlocal x`, then x belongs to an enclosing function, and is neither local nor global
3. If you've found `x = 5` or `for x in range(3)` or some other binding, then x is a **local** variable
4. Otherwise x belongs to some enclosing scope (function scope, global scope, or builtins)

In [11]:
a = 'global'

class Fred:
    a = 'class'  # class scope
    b = (a for i in range(10))  # function scope
    c = [a for i in range(10)]  # function scope
    d = a  # class scope
    e = lambda: a  # function scope
    f = lambda a=a: a  # default argument uses class scope
    
    @staticmethod  # or @classmethod, or regular instance method
    def g():  # function scope
        return a
    
print(Fred.a) # class
print(next(Fred.b)) # global
print(Fred.c[0]) # class in Python 2, global in Python 3
print(Fred.d) # class
print(Fred.e()) # global
print(Fred.f()) # class
print(Fred.g()) # global

class
global
global
class
global
class
global


In [16]:
#del a

class A:
    a = 42
    b = list(a + i for i in range(10))
    
print(A.b)

NameError: name 'a' is not defined

In [18]:
import dis
def foo():
    class A:
        a = 42
        b = list(a + i for i in range(10))
    return A
dis.dis(foo)

  3           0 LOAD_BUILD_CLASS
              2 LOAD_CONST               1 (<code object A at 0x0000013D4261D660, file "<ipython-input-18-ac3f4ec098ee>", line 3>)
              4 LOAD_CONST               2 ('A')
              6 MAKE_FUNCTION            0
              8 LOAD_CONST               2 ('A')
             10 CALL_FUNCTION            2
             12 STORE_FAST               0 (A)

  6          14 LOAD_FAST                0 (A)
             16 RETURN_VALUE


In [19]:
foo.__code__.co_consts

(None,
 <code object A at 0x0000013D4261D660, file "<ipython-input-18-ac3f4ec098ee>", line 3>,
 'A')

In [20]:
dis.dis(foo.__code__.co_consts[1])

  3           0 LOAD_NAME                0 (__name__)
              2 STORE_NAME               1 (__module__)
              4 LOAD_CONST               0 ('foo.<locals>.A')
              6 STORE_NAME               2 (__qualname__)

  4           8 LOAD_CONST               1 (42)
             10 STORE_NAME               3 (a)

  5          12 LOAD_NAME                4 (list)
             14 LOAD_CONST               2 (<code object <genexpr> at 0x0000013D4261D930, file "<ipython-input-18-ac3f4ec098ee>", line 5>)
             16 LOAD_CONST               3 ('foo.<locals>.A.<genexpr>')
             18 MAKE_FUNCTION            0
             20 LOAD_NAME                5 (range)
             22 LOAD_CONST               4 (10)
             24 CALL_FUNCTION            1
             26 GET_ITER
             28 CALL_FUNCTION            1
             30 CALL_FUNCTION            1
             32 STORE_NAME               6 (b)
             34 LOAD_CONST               5 (None)
             36 RET

In [21]:
foo.__code__.co_consts[1].co_consts

('foo.<locals>.A',
 42,
 <code object <genexpr> at 0x0000013D4261D930, file "<ipython-input-18-ac3f4ec098ee>", line 5>,
 'foo.<locals>.A.<genexpr>',
 10,
 None)

In [22]:
dis.dis(foo.__code__.co_consts[1].co_consts[2])

  5           0 LOAD_FAST                0 (.0)
        >>    2 FOR_ITER                14 (to 18)
              4 STORE_FAST               1 (i)
              6 LOAD_GLOBAL              0 (a)
              8 LOAD_FAST                1 (i)
             10 BINARY_ADD
             12 YIELD_VALUE
             14 POP_TOP
             16 JUMP_ABSOLUTE            2
        >>   18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


In [23]:
foo.__code__.co_cellvars 

()

In [24]:
foo.__code__.co_consts[2].co_cellvars

AttributeError: 'str' object has no attribute 'co_cellvars'

In [25]:
foo.__code__.co_consts[2].co_consts[2].co_freevars

AttributeError: 'str' object has no attribute 'co_consts'

In [26]:
class A:
    a = 42
    def b(x):
        return list(x + i for i in range(10))
    b = b(a)
    
print(A.b)

[42, 43, 44, 45, 46, 47, 48, 49, 50, 51]


In [27]:
foo = 1

def func():
    bar = 2
    print(globals().keys()) # prints all variable names in global scope
    print(locals().keys()) # prints all variable names in local scope
    
func()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'counter', '_i2', 'c', '_2', '_i3', '_3', '_i4', '_i5', 'x', 'read_x', 'read_y', '_i6', '_i7', 'read_x_local_fail', '_i8', 'change_local_x', '_i9', 'change_global_x', '_i10', '_i11', 'Fred', '_i12', '_i13', '_i14', '_i15', '_i16', '_i17', 'dis', '_i18', 'foo', '_i19', '_19', '_i20', '_i21', '_21', '_i22', '_i23', '_23', '_i24', '_i25', '_i26', 'A', '_i27', 'func'])
dict_keys(['bar'])


In [29]:
def func():
    foo = 2 # creates a new variable foo in local scope, global foo is not affected
    print(foo) # prints 2
    
    # global variable foo still exists, unchanged:
    print(globals()['foo']) # prints 1
    print(locals()['foo']) # prints 2
    
func()

2
1
2


In [33]:
def func():
    # In this function, foo is a global variable from the beginning
    global foo
    foo = 7 # global foo is modified
    
    print(foo) # 7
    print(globals()['foo']) # 7
    
    #global foo # this could be anywhere within the function
    #print(foo) # 7
    
func()

7
7


In [39]:
def f1():
    
    def f2():
        foo = 2 # a new foo local in f2
        
        def f3():
            nonlocal foo # foo from f2, which is the nearest enclosing scope
            print(foo) # 2
            foo = 20 # modifies foo from f2!
        
        return f3
            
    return f2
            
f1()()()

2
2
